<a href="https://colab.research.google.com/github/neerdwivedi/Jupiter_Notebook/blob/main/ML_Intel_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install numpy==1.24.3 pandas==2.0.3 scikit-learn==1.3.0 tqdm

In [ ]:
import os
import shutil

# Create directories
!mkdir -p data/human_code data/ai_code

# Copy files from existing directories
print("Copying files from existing directories...")

# Copy human code files
!cp "Human Code"/*.py data/human_code/

# Copy AI code files
!cp "AI Code"/*.py data/ai_code/

# Show what files were copied
print("\nFiles in data/human_code:")
!ls -l data/human_code
print("\nFiles in data/ai_code:")
!ls -l data/ai_code

# Count files
human_count = len([f for f in os.listdir('data/human_code') if f.endswith('.py')])
ai_count = len([f for f in os.listdir('data/ai_code') if f.endswith('.py')])
print(f"\nTotal files loaded: {human_count} human code files, {ai_count} AI code files")

Copying files from existing directories...
cp: cannot stat 'Human Code/*.py': No such file or directory
cp: cannot stat 'AI Code/*.py': No such file or directory

Files in data/human_code:
total 68
-rw-r--r-- 1 root root  1717 Apr 20 13:23  ball_bounce.py
-rw-r--r-- 1 root root   437 Apr 20 13:23  battery.py
-rw-r--r-- 1 root root 23779 Apr 20 13:23  biling_system.py
-rw-r--r-- 1 root root   165 Apr 20 13:23  converter_terminal.py
-rw-r--r-- 1 root root   491 Apr 20 13:23  decimal_to_binary.py
-rw-r--r-- 1 root root  3236 Apr 20 13:23  detect_translate.py
-rw-r--r-- 1 root root  1145 Apr 20 13:23  duplicatefileremover.py
-rw-r--r-- 1 root root  1422 Apr 20 13:23  password_generator.py
-rw-r--r-- 1 root root  1270 Apr 20 13:23 'Plagiarism Checker.py'
-rw-r--r-- 1 root root  2342 Apr 20 13:23  rock-paper-scissors.py
-rw-r--r-- 1 root root    90 Apr 20 13:19  test1.py
-rw-r--r-- 1 root root   146 Apr 20 13:19  test2.py

Files in data/ai_code:
total 48
-rw-r--r-- 1 root root 415 Apr 20 13:

In [ ]:
%%writefile code_classifier.py
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import re
from tqdm.notebook import tqdm

class CodeClassifier:
    def __init__(self):
        self.traditional_model = None
        self.vectorizer = None
        self.supported_extensions = ['.py', '.js', '.java', '.cpp', '.c', '.html', '.css']

    def preprocess_code(self, code_content):
        """Preprocess code by removing comments, extra whitespace, etc."""
        # Remove comments
        code_content = re.sub(r'#.*$', '', code_content, flags=re.MULTILINE)  # Python comments
        code_content = re.sub(r'//.*$', '', code_content, flags=re.MULTILINE)  # Single line comments
        code_content = re.sub(r'/\*.*?\*/', '', code_content, flags=re.DOTALL)  # Multi-line comments

        # Remove empty lines and normalize whitespace
        lines = [line.strip() for line in code_content.split('\n')]
        lines = [line for line in lines if line]
        code_content = '\n'.join(lines)

        return code_content

    def load_github_data(self, human_code_path='data/human_code'):
        """Load human-written code from local directory"""
        human_code = []
        file_count = 0
        error_count = 0

        # Walk through all code files in the human code directory
        for root, _, files in os.walk(human_code_path):
            for file in files:
                if any(file.endswith(ext) for ext in self.supported_extensions):
                    try:
                        with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                            code_content = f.read()
                            if len(code_content.strip()) < 10:
                                continue
                            code_content = self.preprocess_code(code_content)
                            human_code.append(code_content)
                            file_count += 1
                    except Exception as e:
                        print(f"Error reading file {file}: {e}")
                        error_count += 1
                        continue

        print(f"Loaded {len(human_code)} human code samples")
        return human_code

    def load_ai_generated_data(self, ai_code_path='data/ai_code'):
        """Load AI-generated code from local directory"""
        ai_code = []
        file_count = 0
        error_count = 0

        # Walk through all code files in the AI code directory
        for root, _, files in os.walk(ai_code_path):
            for file in files:
                if any(file.endswith(ext) for ext in self.supported_extensions):
                    try:
                        with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                            code_content = f.read()
                            if len(code_content.strip()) < 10:
                                continue
                            code_content = self.preprocess_code(code_content)
                            ai_code.append(code_content)
                            file_count += 1
                    except Exception as e:
                        print(f"Error reading file {file}: {e}")
                        error_count += 1
                        continue

        print(f"Loaded {len(ai_code)} AI code samples")
        return ai_code

    def prepare_data(self, human_code_path='data/human_code', ai_code_path='data/ai_code'):
        """Prepare and combine both human and AI code"""
        human_code = self.load_github_data(human_code_path)
        ai_code = self.load_ai_generated_data(ai_code_path)

        if not human_code or not ai_code:
            raise ValueError("No code samples found. Please check your data directories.")

        # Combine data
        code_samples = human_code + ai_code
        labels = [0] * len(human_code) + [1] * len(ai_code)

        return code_samples, labels

    def train_traditional_model(self, code_samples, labels):
        """Train the traditional ML model using TF-IDF and Logistic Regression"""
        # Enhanced TF-IDF with better parameters for code
        self.vectorizer = TfidfVectorizer(
            max_features=2000,
            ngram_range=(1, 3),
            token_pattern=r'(?u)\b\w+\b|[^\w\s]',
            strip_accents='unicode',
            lowercase=True
        )

        X = self.vectorizer.fit_transform(code_samples)
        y = np.array(labels)

        # Split with stratification
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        # Use balanced class weights
        self.traditional_model = LogisticRegression(
            random_state=42,
            class_weight='balanced',
            max_iter=1000
        )
        self.traditional_model.fit(X_train, y_train)

        # Evaluate
        y_pred = self.traditional_model.predict(X_test)
        print("\nModel Performance:")
        print(classification_report(y_test, y_pred))

    def predict(self, code_snippet):
        """Make predictions using the model"""
        # Preprocess the input code
        code_snippet = self.preprocess_code(code_snippet)

        # Traditional model prediction
        code_vector = self.vectorizer.transform([code_snippet])
        prediction = self.traditional_model.predict(code_vector)[0]
        probability = self.traditional_model.predict_proba(code_vector)[0]

        # Get feature importance
        feature_scores = pd.DataFrame({
            'feature': self.vectorizer.get_feature_names_out(),
            'importance': code_vector.toarray()[0] * self.traditional_model.coef_[0]
        })
        top_features = feature_scores.nlargest(5, 'importance')

        return {
            "prediction": "AI-Generated" if prediction == 1 else "Human-Written",
            "confidence": f"{max(probability):.2%}",
            "top_features": top_features.to_dict('records')
        }

Overwriting code_classifier.py


In [ ]:
from code_classifier import CodeClassifier
import pandas as pd
from IPython.display import display

# Initialize classifier
classifier = CodeClassifier()

try:
    # Prepare and train models
    code_samples, labels = classifier.prepare_data()
    print(f"\nTotal samples loaded: {len(code_samples)}")

    # Train model
    classifier.train_traditional_model(code_samples, labels)

    # Test with different code snippets
    test_codes = [
        """
        def hello_world():
            print("Hello, World!")
        """,
        """
        import tensorflow as tf
        def create_model():
            return tf.keras.Sequential([
                tf.keras.layers.Dense(128, activation='relu'),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
        """
    ]

    for i, test_code in enumerate(test_codes, 1):
        print(f"\n=== Test Case {i} ===")
        print("Code:")
        print("```python")
        print(test_code.strip())
        print("```")

        result = classifier.predict(test_code)
        print(f"\nPrediction: {result['prediction']}")
        print(f"Confidence: {result['confidence']}")
        print("\nTop contributing features:")
        feature_df = pd.DataFrame(result['top_features'])
        display(feature_df)

except Exception as e:
    print(f"Error: {e}")
    print("Please check your data directories")

Loaded 12 human code samples
Loaded 12 AI code samples

Total samples loaded: 24

Model Performance:
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.50      0.67         2

    accuracy                           0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5


=== Test Case 1 ===
Code:
```python
def hello_world():
            print("Hello, World!")
```

Prediction: AI-Generated
Confidence: 51.26%

Top contributing features:


,feature,importance
0,"""",0.063613
1,""" )",0.030705
2,"( """,0.024305
3,print,0.016243
4,print (,0.016243



=== Test Case 2 ===
Code:
```python
import tensorflow as tf
        def create_model():
            return tf.keras.Sequential([
                tf.keras.layers.Dense(128, activation='relu'),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
```

Prediction: AI-Generated
Confidence: 52.64%

Top contributing features:


,feature,importance
0,", activation",0.021584
1,", activation =",0.021584
2,activation,0.021584
3,activation =,0.021584
4,activation = ',0.021584
